In [2]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [3]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [6]:
time = pd.DataFrame({"time" : pd.date_range("2021-01-01", "2021-01-01 23:59:59", freq="S")})
time["hour"] = time["time"].dt.time
time["minute"] = time["time"].dt.minute
time["second"] = time["time"].dt.second
time["time"] = time["time"].dt.time
time["time_id"] = range(1, len(time) + 1)
time["saved"] = pd.to_datetime("now")

time

C:\Users\Trabajo\AppData\Local\Temp\ipykernel_47496\2371963413.py:1: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  time = pd.DataFrame({"time" : pd.date_range("2021-01-01", "2021-01-01 23:59:59", freq="S")})


,time,hour,minute,second,time_id,saved
0,00:00:00,00:00:00,0,0,1,2024-10-21 16:19:53.808295
1,00:00:01,00:00:01,0,1,2,2024-10-21 16:19:53.808295
2,00:00:02,00:00:02,0,2,3,2024-10-21 16:19:53.808295
3,00:00:03,00:00:03,0,3,4,2024-10-21 16:19:53.808295
4,00:00:04,00:00:04,0,4,5,2024-10-21 16:19:53.808295
...,...,...,...,...,...,...
86395,23:59:55,23:59:55,59,55,86396,2024-10-21 16:19:53.808295
86396,23:59:56,23:59:56,59,56,86397,2024-10-21 16:19:53.808295
86397,23:59:57,23:59:57,59,57,86398,2024-10-21 16:19:53.808295
86398,23:59:58,23:59:58,59,58,86399,2024-10-21 16:19:53.808295


In [8]:
time.to_sql("dim_time", engineDestination, index=False, if_exists="replace")



400